In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/ColabNotebooks/BA/Experiment/vgg-face-pytorch
!pip install -r requirements.txt

/content/gdrive/MyDrive/ColabNotebooks/BA/Experiment/vgg-face-pytorch
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 30.8 MB/s 
     |████████████████████████████████| 592 kB 61.9 MB/s 
     |████████████████████████████████| 2.3 MB 24.9 MB/s 
     |████████████████████████████████| 3.1 MB 61.1 MB/s 
     |████████████████████████████████| 1.0 MB 51.0 MB/s 
  Created wheel for torchfile: filename=torchfile-0.1.0-py3-none-any.whl size=5709 sha256=49ed38bab02cabf316a3bcf304637edf2d6d9cac08df0d0ae8ef4646c24dcf31
  Stored in directory: /root/.cache/pip/wheels/ac/5c/3a/a80e1c65880945c71fd833408cd1e9a8cb7e2f8f37620bb75b
Successfully built torchfile
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling

### Dodging

In [ ]:
%cd /content/gdrive/MyDrive/ColabNotebooks/BA/Experiment/vgg-face-pytorch

import torch
import torch.nn as nn
import torch.optim as optim
from models.utils import preprocess_data
import numpy as np
import torchvision
import cv2
from torchvision import datasets, models, transforms
import sys
import numpy
from models.vgg_face import get_pretrained_model, get_prediction
from image_utils import save_image, show_image 
from glass_attack import glass_attack 
import torch.nn.functional as F
import logging
import math

numpy.set_printoptions(threshold=sys.maxsize)


logger = logging.getLogger("glass_attack_dodging-logger")

logging.basicConfig(
    level=logging.INFO,
    filename="logs/glass_attack_dodging.log",
    filemode="w",
    format="%(asctime)s %(name)s %(levelname)s: %(message)s"
)


names = [line.rstrip('\n') for line in open('data/names.txt')]
 
BATCH_SIZE = 100
NUM_PASSES = 5
ITERA = [1, 2, 3, 5, 7, 10, 15, 20, 30]
ALPHA = 20

model = get_pretrained_model()

model.eval()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model = model.to(device)

glass = cv2.imread('data/glasses/silhouette.png')

glasses = transforms.ToTensor()(glass)
glasses = glasses.to(device)

for i in range(len(ITERA)):

  logger.info(f"Num of attack iterations: {ITERA[i]}")

  dataloaders, dataset_sizes = preprocess_data(BATCH_SIZE)
  dataiter = iter(dataloaders)

  for j in range(NUM_PASSES):

    logger.info(f"Run {j+1}/{NUM_PASSES}")
    data = dataiter.next()

    images, labels = data
    images = images[:,[2,1,0],:,:] #rgb to bgr

    images = images.to(device)
    labels = labels.to(device)
   
    x_adv = glass_attack(model, images, labels, glasses, alpha=ALPHA, num_iter=ITERA[i])

    values, indices = get_prediction(x_adv, model)

    predictions = [(f"id: {index}", f"name: {names[index]}", f"confidence: {value}") for index, value in zip(indices.tolist(), values.tolist())]
    ground_truth = [(f"id: {label}", f"name: {names[label]}") for label in labels.tolist()]  
    
    logger.info(f"\n\nPredictions: {predictions}\n\n"\
                f"Labels: {ground_truth}\n\n" \
                f"Num. predictions: {labels.size(0)}\n" \
                f"Num. correct predictions: {(indices == labels).sum().item()}\n\n")
    
  save_image(f"glass_attack/dodging/iters{ITERA[i]}-", x_adv)
  

### Impersonation

In [ ]:
%cd /content/gdrive/MyDrive/ColabNotebooks/BA/Experiment/vgg-face-pytorch

import torch
import torch.nn as nn
import torch.optim as optim
from models.utils import preprocess_data
import numpy as np
import torchvision
import cv2
from torchvision import datasets, models, transforms
import sys
import numpy
from models.vgg_face import get_pretrained_model, get_prediction
from image_utils import save_image, show_image 
from glass_attack import glass_attack 
import torch.nn.functional as F
import logging
import math
import random

numpy.set_printoptions(threshold=sys.maxsize)


logger = logging.getLogger("glass_attack_impersonation-logger")

logging.basicConfig(
    level=logging.INFO,
    filename="logs/glass_attack_impersonation.log",
    filemode="w",
    format="%(asctime)s %(name)s %(levelname)s: %(message)s"
)


names = [line.rstrip('\n') for line in open('data/names.txt')]
 
BATCH_SIZE = 30
NUM_PASSES = 5
ITERA = 30
ALPHA = 20
IMPERSONATION_ATTEMPTS = 3


model = get_pretrained_model()

model.eval()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model = model.to(device)

glass = cv2.imread('data/glasses/silhouette.png')

glasses = transforms.ToTensor()(glass)
glasses = glasses.to(device)

for i in range(IMPERSONATION_ATTEMPTS):
  target = random.randint(0, 2621)
  target_tensor = torch.full((BATCH_SIZE,), target).to(device)

  logger.info(f"Attempt: {i + 1} - Target: (id: {target}, name: {names[target]})")

  logger.info(f"Num of attack iterations: {ITERA}")

  dataloaders, dataset_sizes = preprocess_data(BATCH_SIZE)
  dataiter = iter(dataloaders)

  for j in range(NUM_PASSES):

    logger.info(f"Run {j+1}/{NUM_PASSES}")
    data = dataiter.next()

    images, labels = data
    images = images[:,[2,1,0],:,:] #rgb to bgr

    images = images.to(device)
    labels = labels.to(device)

    x_adv = glass_attack(model, images, labels, glasses, alpha=ALPHA, num_iter=ITERA, y_target=target_tensor)

    values, indices = get_prediction(x_adv, model)

    predictions = [(f"id: {index}", f"name: {names[index]}", f"confidence: {value}") for index, value in zip(indices.tolist(), values.tolist())]
    ground_truth = [(f"id: {label}", f"name: {names[label]}") for label in labels.tolist()]

    num_total_predictions = labels.size(0)
    num_correct_predictions = (indices == labels).sum().item()
    num_successful_impersonation = (indices == target_tensor).sum().item()
    num_dodging = num_total_predictions - num_correct_predictions - num_successful_impersonation
    
    logger.info(f"\n\nPredictions: {predictions}\n"\
                f"Labels: {ground_truth}\n\n" \
                f"Num. total predictions: {num_total_predictions}\n\n" \
                f"Num. correct predictions (indices == labels): {num_correct_predictions}\n" \
                f"Num. successful IMPERSONATION predictions (indices == targets): {num_successful_impersonation}\n" \
                f"Num. successful DODGING predictions (indices != labels && indices != targets): {num_dodging}\n\n")
    # show_image(x_adv)  
  save_image(f"glass_attack/impersonation/attempt{i}-iters{ITERA}-", x_adv)